# TCC - Victor Gonçalves
## Instruções de obtenção de dados
Autor: Prof. Dr. Reginaldo Gonçalves Leão Junior


### Verificação do status INMET

Consiste na chamada à função `statusINMET()` que verifica a disponibilidade de séries anuais climatológicas no recurso `/dadoshistoricos` do portal, exibe um compilado deste estatus no terminal e retorna um dicionário com a estrutura:

```
Dict{Int16, String} with n entries:
  "20XX" => "https://portal.inmet.gov.br/uploads/dadoshistoricos/20XX.zip"
```

no qual as chaves são os anos disponíveis na base no formato de um inteiro de 32 bits e os valores uma String contendo o link para download do zip file das séries para o respectivo ano.

Antes de qualquer verificação, deve-se fazer a instalação do pacote em desenvolvimento, ou verificação por atualizações.
Sendo o primeiro uso do pacote em uma determinada instalção deve-se executar as instruções do comentário `i)`, nas demais, atualizações devem ser checadas por meio das instruções do comentário `ii)`. A instrução `using Pkg` é apenas uma chamada ao módulo `Pkg` responsável pela gerência dos pacotes.

In [1]:
# using Pkg
# i) Na primeira execução do pacote
# Pkg.develop("SolarPowerForecastingIFMG")

# ii) A cada nova adição de funcionalidade
# Pkg.update("SolarPowerForecastingIFMG")

Após a instalação ou atualização o pacote deve ser carregado no *kernel* em execução, em seguida a verificação do status no INMET pode ser imediatamente realizada.

In [2]:
using SolarPowerForecastingIFMG: statusINMET, obter_dados
using DataFrames
fonte_dados = statusINMET();

Estrato de dados INMET
Primeiro ano disponível	->	2000
Último ano disponível	->	2022


Após a execução a variável `fonte_dados` é um dicionário que contém os links para a obtenção das tabelas CSV de cada ano.

In [3]:
@show fonte_dados[2000];
@show fonte_dados[2010];
@show fonte_dados[2022];

fonte_dados[2000] = "https://portal.inmet.gov.br/uploads/dadoshistoricos/2000.zip"
fonte_dados[2010] = "https://portal.inmet.gov.br/uploads/dadoshistoricos/2010.zip"
fonte_dados[2022] = "https://portal.inmet.gov.br/uploads/dadoshistoricos/2022.zip"


### Raspagem de Dados
A extração de dados em si se dá por meio da função `obter_dados` cuja documentação pode ser vista àbaixo:

In [4]:
?obter_dados

search:



Recebe:

i) fonte_dados::Dict{Int16, String}:  um dicionário com chaves do tipo Int16, contendo o ano de uma determinada série  histórica do INMET e uma String com o link de download da série no servidor

ii) cidades::Vector{String}: um vetor contendo uma lista das cidades a serem estudadas

iii) [intervtemp::Vector{Int}] ou [intervtemp::UnitRange]:  um vetor contendo uma lista dos anos a serem estudados ou um UnitRang com este  intervalo de estudo.


No exemplo abaixo utilizamos a função `obter_dados` para a `fonte_dados` requerendo estes para a cidade de Formiga e o ano de 2010.
Note como esses parâmetros foram passados na forma de vetores, de forma tal que, mais de uma cidade, ou mesmo, mais que um ano de estudo podem ser selcionados. 

In [5]:
dados = obter_dados(fonte_dados, ["FORMIGA"], [2010]);

Excessive output truncated after 524334 bytes.

In [6]:
dados

SolarPowerForecastingIFMG.SerieCidades(SolarPowerForecastingIFMG.EstruturaDeCaptura[SolarPowerForecastingIFMG.EstruturaDeCaptura("FORMIGA", 2010, 8760×20 DataFrame
  Row │ DATA (YYYY-MM-DD)  HORA (UTC)  PRECIPITA\xc7\xc3O TOTAL, HOR\xc…  PRES ⋯
      │ Dates.Date         Dates.Time  Float64                            Floa ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ 2010-01-01         00:00:00                                  0.0       ⋯
    2 │ 2010-01-01         01:00:00                                  0.0
    3 │ 2010-01-01         02:00:00                                  0.0
    4 │ 2010-01-01         03:00:00                                  0.0
    5 │ 2010-01-01         04:00:00                                  0.0       ⋯
    6 │ 2010-01-01         05:00:00                                  0.0
    7 │ 2010-01-01         06:00:00                                  0.0
    8 │ 2010-01-01         07:00:00                               

Quando `dados` é exibido, se vê um problema de codificação no nome das colunas como por exemplo `PRECIPITA\xc7\xc3O` na terceira coluna, isto é uma falha de codificação na fonte dos dados. 
A lista de nomes das colunas pode ser inspecioanda utilizando-se a função name no objeto do tipo `dataset` pertencente à `mutable struct EstruturaDeCaptura`.

In [7]:
names(dados.serie[1].dataset)

20-element Vector{String}:
 "DATA (YYYY-MM-DD)"
 "HORA (UTC)"
 "PRECIPITA\xc7\xc3O TOTAL, HOR\xc1RIO (mm)"
 "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
 "PRESS\xc3O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)"
 "PRESS\xc3O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)"
 "RADIACAO GLOBAL (KJ/m\xb2)"
 "TEMPERATURA DO AR - BULBO SECO, HORARIA (\xb0C)"
 "TEMPERATURA DO PONTO DE ORVALHO (\xb0C)"
 "TEMPERATURA M\xc1XIMA NA HORA ANT. (AUT) (\xb0C)"
 "TEMPERATURA M\xcdNIMA NA HORA ANT. (AUT) (\xb0C)"
 "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (\xb0C)"
 "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (\xb0C)"
 "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)"
 "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)"
 "UMIDADE RELATIVA DO AR, HORARIA (%)"
 "VENTO, DIRE\xc7\xc3O HORARIA (gr) (\xb0 (gr))"
 "VENTO, RAJADA MAXIMA (m/s)"
 "VENTO, VELOCIDADE HORARIA (m/s)"
 "Column20"

A estrtura de captura de dados no submódulo `RaspagemDeDadosINMET.jl` é organizada na forma:
```Julia
mutable struct EstruturaDeCaptura
    cidade::Union{String, Nothing}
    ano::Union{Int, String, Nothing}
    dataset::Union{DataFrame, Nothing}
end

mutable struct SerieCidades 
    serie::AbstractVector{EstruturaDeCaptura}
end
```
Na qual a `mutable struct SerieCidades` é utilizada para armazenar os dados das múltiplas cidades desejadas para o estudo, e a `mutable struct EstruturaDeCaptura` cotém o nome de cada cidade, e o nome de cada ano de estudo, além dos dados proprieamente ditos na forma de `DataFrame` do tipo `DataFrames.jl`.

Em nosso caso, as seguintes colunas precisam de ajuste de codificação de caractere:

3. `PRECIPITA\xc7\xc3O TOTAL, HOR\xc1RIO (mm)` para `PRECIPITACAO TOTAL, HORARIA (mm)`;
5. `PPRESS\xc3O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)` para `PRESSAO ATMOSFERICA MAX.NA HORA ANT.(AUT) (mB)`;
6. `PPRESS\xc3O ATMOSFERICA MIN.NA HORA ANT. (AUT) (mB)` para `PRESSAO ATMOSFERICA MIN.NA HORA ANT.(AUT) (mB)`;
7. `RADIACAO GLOBAL (KJ/m\xb2)` para `RADIACAO GLOBAL (KJ/m²)`;
8. `TEMPERATURA DO AR - BULBO SECO, HORARIA (\xb0C)` para `TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)`;
9. `TEMPERATURA DO PONTO DE ORVALHO (\xb0C)` para `TEMPERATURA DO PONTO DE ORVALHO (°C)`;
10. `TEMPERATURA M\xc1XIMA NA HORA ANT. (AUT) (\xb0C)` para `TEMPERATURA MAXIMA NA HORA ANT.(AUT) (°C)`;
11. `TEMPERATURA M\xcdNIMA NA HORA ANT. (AUT) (\xb0C)` para `TEMPERATURA MINIMA NA HORA ANT.(AUT) (°C)`;
12. `TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (\xb0C)` para `TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)`;
13. `TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (\xb0C)` para `TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)`;
18. `VENTO, DIRE\xc7\xc3O HORARIA (gr) (\xb0 (gr))` para `VENTO, DIRECAO HORARIA (gr) (°(gr))`;

Para automatizar o processo cria-se um dicionário com os índices das colunas a serem modificadas e os novos nomes seguido do ajuste iterativo sobre `dados.serie[1].dataset`

In [8]:
columns_change = (
    3=>"PRECIPITACAO TOTAL, HORARIA (mm)",
    5=>"PRESSAO ATMOSFERICA MAX.NA HORA ANT.(AUT) (mB)",
    6=>"PRESSAO ATMOSFERICA MIN.NA HORA ANT.(AUT) (mB)",
    7=>"RADIACAO GLOBAL (KJ/m²)",
    8=>"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
    9=>"TEMPERATURA DO PONTO DE ORVALHO (°C)",
    10=>"TEMPERATURA MAXIMA NA HORA ANT.(AUT) (°C)",
    11=>"TEMPERATURA MINIMA NA HORA ANT.(AUT) (°C)",
    12=>"TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)",
    13=>"TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)",
    18=>"VENTO, DIRECAO HORARIA (gr) (°(gr))"
)
for col in columns_change
       rename!(dados.serie[1].dataset, col)
end

Evento que corrije o problema de codificação conforme pode ser verificado pelo código abaixo. Alguma abordagem funcional usando `map` ou `foreach` também pode ser usada, inclusive talvez até com maior desempenho. 

In [10]:
names(dados.serie[1].dataset)

20-element Vector{String}:
 "DATA (YYYY-MM-DD)"
 "HORA (UTC)"
 "PRECIPITACAO TOTAL, HORARIA (mm)"
 "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
 "PRESSAO ATMOSFERICA MAX.NA HORA ANT.(AUT) (mB)"
 "PRESSAO ATMOSFERICA MIN.NA HORA ANT.(AUT) (mB)"
 "RADIACAO GLOBAL (KJ/m²)"
 "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"
 "TEMPERATURA DO PONTO DE ORVALHO (°C)"
 "TEMPERATURA MAXIMA NA HORA ANT.(AUT) (°C)"
 "TEMPERATURA MINIMA NA HORA ANT.(AUT) (°C)"
 "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)"
 "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)"
 "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)"
 "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)"
 "UMIDADE RELATIVA DO AR, HORARIA (%)"
 "VENTO, DIRE\xc7\xc3O HORARIA (gr) (\xb0 (gr))"
 "VENTO, DIRECAO HORARIA (gr) (°(gr))"
 "VENTO, VELOCIDADE HORARIA (m/s)"
 "Column20"

## Aquisições múltiplas
A função `obter_dados` ainda possibilita a aquisição múltipla de dados, tanto para múltiplas cidades, quanto para distintos intervalos de tempo, veja os exemplos abaixos.

### Aquisição de dados para múltiplas cidades
Na caso da aquisição para várias cidades basta declarar quais aquelas desejadas para estudo no segundo parâmetro da função, como no exemplo abaixo. É importante considerar que as cidades declaras devem constar dentre aquelas onde o INMET mantém coleta de dados. Futuramente a função `statusINMET` checará essa informações de maneira automatizada para disponibilizar aos usuários.

In [13]:
dados = obter_dados(fonte_dados, ["FORMIGA", "BAMBUI"], [2020]);

Excessive output truncated after 524342 bytes.

Neste caso veja como os dados ficam organizados em `dados`.
Como já dito a variável é do tipo `SerieCidades` e possui o campo `serie` que contém um vetor do tipo `Vector{SolarPowerForecastingIFMG.EstruturaDeCaptura}`.

In [15]:
@show typeof(dados)
@show typeof(dados.serie)

typeof(dados) = SolarPowerForecastingIFMG.SerieCidades
typeof(dados.serie) = Vector{SolarPowerForecastingIFMG.EstruturaDeCaptura}


Vector{EstruturaDeCaptura} (alias for Array{SolarPowerForecastingIFMG.EstruturaDeCaptura, 1})

O comprimento do vetor armazenado no campo `serie` pode ser exibido fazendo-se:

In [16]:
length(dados.serie)

2

Onde o primeiro elemento se refere à primeira cidade declarada em `obter_dados` e o segundo à segunda.

### ATENÇÃO: A função `obter_dados` não está organizando corretamente os dados de distintas cidades, além disso é preciso verificar como organizar esses dados para múltiplas cidades e múltiplos anos. Corrigido isso também explicar como a aquisição se dá via vetor com múltiplos anos ou usando UnitRanges.

In [20]:
println("Primeira cidade: ", dados.serie[1].cidade)
println("Ano da série: ", dados.serie[1].ano)

println("Segunda cidade: ", dados.serie[2].cidade)
println("Ano da série: ", dados.serie[2].ano)

Primeira cidade: BAMBUI
Ano da série: 2020
Segunda cidade: BAMBUI
Ano da série: 2020
